In [ ]:
import torch
import pandas as pd
import pickle

from torch.utils.data import DataLoader

import src.models
import src.preprocess
import src.dataset
import src.utils
import src.eval
import src.train

from src.config import *

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
LOAD = True

# LSTM

In [ ]:
encoder = src.preprocess.OneHotEncoder(TOKENS)

df_test = pd.read_csv("data/test.csv", index_col="seq_id")
df_grouped = pd.read_csv("data/train_cleaned_grouped.csv", index_col="seq_id")

train_group_datasets, val_group_datasets, val_idxs = src.dataset.get_preprocessed_grouped_datasets(
    df_grouped,
    encoder,
    test_size=0.3
)

train_dataset, test_dataset = src.dataset.get_preprocessed_datasets(
    df_grouped,
    df_test,
    val_idxs,
    encoder
)

In [ ]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=src.dataset.pad_collate_train
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=False,
    collate_fn=src.dataset.pad_collate_test
)

train_group_loaders = []
train_group_loaders2 = []
val_group_loaders = []

for dataset in train_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                            batch_size=1,
                            shuffle=True)
    train_group_loaders.append(data_loader)

for dataset in train_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                             batch_size=len(dataset),
                             shuffle=False,
                             collate_fn=src.dataset.pad_collate_train)
    train_group_loaders2.append(data_loader)    

for dataset in val_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                             batch_size=len(dataset),
                             shuffle=False,
                             collate_fn=src.dataset.pad_collate_train)
    val_group_loaders.append(data_loader)

In [ ]:
with open("output/lstm_train_group_datasets.pkl", "wb") as f:
    pickle.dump(train_group_datasets, f)
with open("output/lstm_val_group_datasets.pkl", "wb") as f:
    pickle.dump(val_group_datasets, f)

In [ ]:
model = src.models.RNN(128, num_layers=2).to(device)

In [ ]:
losses, val_scores = src.train.train_model(
    model, 
    train_loader, 
    train_group_loaders,
    train_group_loaders2,
    val_group_loaders,
    NUM_EPOCHS, 
    LEARNING_RATE,
    "models/lstm.ckpt",
    True,
    device
)

src.utils.save_losses(losses, "output/lstm_losses.csv")
src.utils.save_scores(val_scores, "output/lstm_val_scores.csv")

In [ ]:
rnn_model = src.models.RNN(128, 2).to(device)
rnn_model.load_state_dict(torch.load("models/lstm.ckpt"))
src.eval.generate_test_output(rnn_model, test_loader, "output/lstm.csv", device)

# Transformer

In [ ]:
encoder = src.preprocess.NumericEncoder(TOKENS)

df_test = pd.read_csv("data/test.csv", index_col="seq_id")
df_grouped = pd.read_csv("data/train_cleaned_grouped.csv", index_col="seq_id")

train_group_datasets, val_group_datasets, val_idxs = src.dataset.get_preprocessed_grouped_datasets(
    df_grouped,
    encoder,
    test_size=0.3
)

train_dataset, test_dataset = src.dataset.get_preprocessed_datasets(
    df_grouped,
    df_test,
    val_idxs,
    encoder
)

In [ ]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=src.dataset.pad_collate_train
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE, 
    shuffle=False,
    collate_fn=src.dataset.pad_collate_test
)

train_group_loaders = []
train_group_loaders2 = []
val_group_loaders = []

for dataset in train_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                            batch_size=1,
                            shuffle=True)
    train_group_loaders.append(data_loader)
    
for dataset in train_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                             batch_size=len(dataset),
                             shuffle=False,
                             collate_fn=src.dataset.pad_collate_train)
    train_group_loaders2.append(data_loader)   

for dataset in val_group_datasets:
    data_loader = DataLoader(dataset=dataset,
                             batch_size=len(dataset),
                             shuffle=False,
                             collate_fn=src.dataset.pad_collate_train)
    val_group_loaders.append(data_loader)

In [ ]:
with open("output/tf_train_group_datasets.pkl", "wb") as f:
    pickle.dump(train_group_datasets, f)
with open("output/tf_val_group_datasets.pkl", "wb") as f:
    pickle.dump(val_group_datasets, f)

In [ ]:
model = src.models.TransformerEncoderModel(nlayers=2).to(device)

In [ ]:
losses, val_scores = src.train.train_model(
    model, 
    train_loader, 
    train_group_loaders,
    train_group_loaders2,
    val_group_loaders,
    NUM_EPOCHS, 
    LEARNING_RATE,
    "models/transformer.ckpt",
    True,
    device
)

src.utils.save_losses(losses, "output/transformer_losses.csv")
src.utils.save_scores(val_scores, "output/transformer_val_scores.csv")

In [ ]:
tf_model = src.models.TransformerEncoderModel(nlayers=2).to(device)
tf_model.load_state_dict(torch.load("models/transformer.ckpt"))
src.eval.generate_test_output(tf_model, test_loader, "output/transformer.csv", device)